In [1]:
import pandas as pd
from sklearn import linear_model

#read file
df = pd.read_excel('DataOnly\\WaitData.Published.xlsx')

Y = df['Wait']
X = df.drop(['Wait', 'x_ScheduledDTTM', 'x_ArrivalDTTM', 'x_BeginDTTM'], 1)

In [5]:
#linear regression
model = linear_model.LinearRegression()
model.fit(X, Y)
Ypred = model.predict(X)
r = Y-Ypred
e = abs(r).mean()

print(e)

23.16628893634752


In [6]:
# Run Python feature selection
if True: # just in case I want to disable this part
    print('\n>Python feature selection:')
    from sklearn.feature_selection import RFE
    from itertools import compress
    for nFeatures in range(1,4):
        rfe = RFE(model, n_features_to_select=nFeatures)
        X_rfe = rfe.fit_transform(X,Y) #transforming data using RFE
        #Fitting the data to model
        model.fit(X_rfe,Y)
        #print(rfe.support_)
        #print(rfe.ranking_)
        cols = list(compress( X.columns, rfe.support_))
        model.fit(X[cols],Y)
        e = abs(Y-model.predict(X[cols])).mean()
        print(e, cols)


>Python feature selection:
32.50541172120655 ['CardiacCount']
31.224937706982736 ['CardiacCount', 'DelayedInLine']
31.25752894868932 ['CardiacCount', 'StartTime', 'DelayedInLine']


In [9]:
import numpy as np
import sys

def lin_reg(X, Y):
    model = linear_model.LinearRegression()
    model.fit(X, Y)
    Ypred = model.predict(X) # use trained regression model to predict
    r = Y-Ypred # compute prediction error (residual)
    return abs(r).mean() #

min_e = sys.maxsize
best_column = ''


31.227057699733514


In [14]:
X_nf = X
for n in range(0, 15): # n is the number of features
    min_e = sys.maxsize
    for column in X_nf.columns:
        if n == 0:
            X_new = np.array(X[column].values.tolist())
            X_new = X_new.reshape(-1, 1)
        else:
            X_new = pd.concat([X_best, X_nf[column]], axis=1)
        e_new = lin_reg(X_new, Y)
        if min_e > e_new:
            best_column = column
            min_e = e_new

    if n == 0:
        X_best = X[best_column]
        X_nf = X.drop(best_column, axis='columns')
    else:
        X_best = pd.concat([X_best, X[best_column]], axis=1)
        X_nf = X_nf.drop(best_column, axis='columns')
    print(n+1, min_e, best_column)

1 31.227057699733514 DelayedInLine
2 30.137965863467567 LineCount0Strict
3 28.69649063225338 LineCount0
4 28.2463095851323 SumDelayWaitingInLine
5 28.147006842164515 AheadCount
6 24.989574734774777 NumCompletedToday
7 24.659186407251333 NumScheduledNextW2
8 24.329071006187288 DelayCount
9 24.071936786758197 InProgressSize
10 23.84650949206529 StartTime4
11 23.692619398547166 AvgDelayForDay
12 23.627914296582315 AvgWaitLastK1Customers
13 23.57915967047999 SumDelayInProgress
14 23.548081793881664 NumScheduledNextSlot
15 23.52096351255369 IsFirst


In [15]:
all_variants = []
X_best = X['LineCount0']
X_nf = X

for f1 in X.columns:
    X_2f = X.drop(f1, axis='columns')
    for f2 in X_2f.columns:
        X_3f = X_2f.drop(f2, axis='columns')
        for f3 in X_3f.columns:
            X_new = pd.concat([X[f1], X_2f[f2], X_3f[f3]], axis=1)
            e_new = lin_reg(X_new, Y)
            all_variants.append((e_new, f1, f2, f3))

all_variants.sort()
print(all_variants[0])

(26.072745167091213, 'NumCompletedToday', 'AheadCount', 'SumDelayWaitingByExamCode')
